In [ ]:
import seaborn as sns
sns.set()

In [ ]:
from static_grader import grader

# SQL Miniproject


## Introduction


The city of New York does restaurant inspections and assigns a grade. Inspections data for the last 4 years are available on s3 as an SQLite database, which you can import in the next few cells. These were extracted from a set of csv files and an XLS file, as described in the <b>How we loaded the data</b> section


The raw data can be found [here](https://s3.amazonaws.com/dataincubator-course/coursedata/nyc_inspection_data.zip) and can be useful to look at. The file `RI_Webextract_BigApps_Latest.xls` contains a description of each of the data files and what the columns mean.

In [ ]:
!aws s3 sync s3://dataincubator-course/coursedata/ . --exclude '*' --include 'nyc_inspection.db'

In [ ]:
#This will load the pre-existing tables
%load_ext sql
%sql sqlite:///nyc_inspection.db

In [ ]:
import sys
import re
import sqlite3

con = sqlite3.connect('nyc_inspection.db')
con.enable_load_extension(True)
con.text_factory = str
c = con.cursor()  

In [ ]:
result = con.execute("SELECT * FROM webextract")

To see what tables are in the database:

In [ ]:
%%sql
SELECT * FROM sqlite_master
WHERE type='table';

And to look at the format of an individual table (note that you may need to change types to get the answers in the right form):

In [ ]:
%%sql
PRAGMA table_info(webextract)

## SQLite3


The project should be written in SQL. Between SQLite and PostgreSQL we recommend SQLite for this project.  You can use the SQLite command prompt by running this command in bash
```bash
sqlite3 cmd "DROP TABLE IF EXISTS writer;\
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);\
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);\
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);\
\
SELECT * FROM writer;\
"
```
Alternatively, you can run bash commands in a Jupyter notebook by prepending the `!` in a code cell (notice that we conveniently get the output displayed

In [ ]:
!sqlite3 cmd """\
DROP TABLE IF EXISTS writer;\
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);\
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);\
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);\
\
SELECT * FROM writer;\
"""

Finally, we use the [`ipython-sql` extension](https://github.com/catherinedevlin/ipython-sql#ipython-sql) by first loading this extension and then running our code with the "magic" command in the first line
```python
%%sql sqlite://
```
Notice that the output table is formatted nicely as a nice HTML table.

This is our recommended technique.  However, the grader is expecting python objects and you may need to use list comprehensions to reformat this output

In [ ]:
%%sql 
DROP TABLE IF EXISTS writer;
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);

SELECT * FROM writer;

In [ ]:
result = _
#This captures the output of the previous cell

In [ ]:
result

## How we loaded the data


For future reference, here is how you can load data in to SQL (with examples).  If you have a csv file you created with something like

```
!printf "Name,Age\nAlice,3\nBob,10" > sample.csv.nogit
```


Then SQLite has a convenient [`.import` function](https://sqlite.org/cli.html#csv_import) which can create tables from `.csv` files.

```bash
sqlite> .import sample.csv.nogit sample
sqlite> SELECT * FROM sample;
```

The files may contain badly formatted text.  Unfortunately, this is all too common.  As a stop gap, remember that [`iconv`](https://linux.die.net/man/1/iconv) is a Unix utility that can convert files between different text encodings.

Alternatively, you can also read csv files using pandas and convert that into SQL via some SQL magic (this is what we actually did).

```
import pandas as pd
sample = pd.read_csv('sample.csv.nogit')
%sql DROP TABLE IF EXISTS sample
%sql PERSIST sample
%sql SELECT * FROM sample;
```

## Question 1: Null entries


Return the number of inspections (`CAMIS`, `INSPDATE` pairs) that do not have a score - i.e. where none of the rows with those (`CAMIS`, `INSPDATE`) values has a score. Remove the corresponding rows from the data set for the rest of the questions in the assignment.

As an example we might see this:

```
SELECT CAMIS, INSPDATE, SCORE FROM webextract
ORDER BY CAMIS, INSPDATE
LIMIT 10
```

| `CAMIS` | `INSPDATE` | `SCORE` |
| --- | --- | --- |
| 30075445 | 2011-03-10 00:00:00 | 14.0 |
| 30075445 | 2011-03-10 00:00:00 | 14.0 |
| 30075445 | 2011-03-10 00:00:00 | 14.0 |
| 30075445 | 2011-03-10 00:00:00 | None |
| 30075445 | 2011-04-27 00:00:00 | None |
| 30075445 | 2011-11-12 00:00:00 | None |
| 30075445 | 2011-11-12 00:00:00 | None |
| 30075445 | 2011-11-23 00:00:00 | 9.0 |
| 30075445 | 2011-11-23 00:00:00 | 9.0 |
| 30075445 | 2011-11-23 00:00:00 | 9.0 |

We want to filter only those dates for a given `CAMIS` where _every_ score is NULL (or `None` in the Python output).  In this case, we would remove 2011-04-27 and 2011-11-12, but keep 2011-03-10 (and 2011-11-23, of course).  Our post-filtering table would be:

| `CAMIS` | `INSPDATE` | `SCORE` |
| --- | --- | --- |
| 30075445 | 2011-03-10 00:00:00 | 14.0 |
| 30075445 | 2011-03-10 00:00:00 | 14.0 |
| 30075445 | 2011-03-10 00:00:00 | 14.0 |
| 30075445 | 2011-03-10 00:00:00 | None |
| 30075445 | 2011-11-23 00:00:00 | 9.0 |
| 30075445 | 2011-11-23 00:00:00 | 9.0 |
| 30075445 | 2011-11-23 00:00:00 | 9.0 |

**Question:** How else might we have handled this?

In [ ]:
%%sql
SELECT * FROM webextract
LIMIT 5

In [ ]:
%%sql
SELECT CAMIS, INSPDATE, count(score) as null_entries FROM webextract
GROUP BY CAMIS, INSPDATE
HAVING count(score) = 0

In [ ]:
result = _

In [ ]:
lst = [x for x in result]
null_entries = 0 
for i in range(len(result)):
    null_entries+=1

In [ ]:
grader.score('sql__null_entries', null_entries)

## Question 2: Score by zip code


Return a list of tuples of the form:

    (zip code, mean score, number of restaurants)

for each of the 87 zip codes in the city with over 100 restaurants. Use the score from the latest inspection date for each restaurant. Sort the list in ascending order by mean score.

**Note:** There is an interesting discussion here about what the mean score *means* in this data set. Think about what we're actually calculating - does it represent what we're trying to understand about these zip codes?

What if we use the average of a restaurant's inspections instead of the latest?

**Checkpoints:**
- Total unique restaurants: 24,361;
- Total restaurants in valid zip codes: 19,172

In [ ]:
%%sql
DROP TABLE IF EXISTS temp_webextract;

CREATE TABLE temp_webextract AS
SELECT * FROM webextract
WHERE CAMIS IN (SELECT CAMIS FROM webextract
GROUP BY CAMIS, INSPDATE
HAVING count(score) != 0);

In [ ]:
%%sql
SELECT count(*) FROM temp_webextract

In [ ]:
%%sql
SELECT SUM(C) FROM
(SELECT COUNT(DISTINCT CAMIS) AS C FROM temp_webextract)

In [ ]:
%%sql
SELECT SUM(c) FROM
(SELECT COUNT(DISTINCT CAMIS) AS c from temp_webextract
GROUP BY zipcode 
HAVING COUNT(DISTINCT CAMIS)>100)

In [ ]:
%%sql
SELECT zipcode, camis, MAX(INSPDATE), score from temp_webextract
GrOUP BY zipcode, Camis
LIMIT 10

In [ ]:
%%sql
SELECT zipcode, AVG(score), COUNT(camis) from 
(SELECT zipcode, score, camis, MAX(INSPDATE) from temp_webextract
GrOUP BY zipcode, Camis)
GROUP BY zipcode 
HAVING COUNT(camis) >100
ORDER BY AVG(score)

In [ ]:
Result = _

In [ ]:
score_by_zipcode = [(str(int(x)), y, z) for x,y,z in Result]

In [ ]:
score_by_zipcode

In [ ]:
# score_by_zipcode = [("11201", 9.81739130434783, 345)] * 87

grader.score('sql__score_by_zipcode', score_by_zipcode)

## Question 3: Score by borough

Return a list of tuples of the form:

    (borough, mean score, number of restaurants)

for each of the city's five boroughs. Use the latest score for each restaurant. Sort the list in ascending order by grade.

**Hint:** You will have to perform a join with the `boroughs` table. The borough names should be reported in ALL CAPS.

**Checkpoint:**
- Total restaurants in valid boroughs: 24,350

In [ ]:
%%sql 
SELECT * FROM temp_webextract
LEFT OUTER JOIN boroughs
ON temp_webextract.boro = boroughs.id
LIMIT 10


In [ ]:
%%sql
SELECT SUM(c) FROM
(SELECT boro, COUNT(DISTINCT CAMIS) AS c from temp_webextract
WHERE boro in ('1', '2', '3', '4', '5')
GROUP BY boro )

In [ ]:
%%sql
SELECT name, AVG(score), COUNT(camis) FROM
(SELECT name, score, camis, MAX(INSPDATE) FROM temp_webextract 
 LEFT OUTER JOIN boroughs
 ON temp_webextract.boro = boroughs.id
GrOUP BY name, Camis)
WHERE name in ("THE BRONX", "MANHATTAN", "BROOKLYN", "STATEN ISLAND", "QUEENS")
GROUP BY name 
ORDER BY AVG(score)

In [ ]:
Res = _

In [ ]:
score_by_borough = [(x, y, z) for x,y,z in Res]

In [ ]:
score_by_borough

In [ ]:
# score_by_borough = [("MANHATTAN", 10.7269875502402, 10201)] * 5
grader.score('sql__score_by_borough', score_by_borough)

## Question 4: Violations by cuisine


We want to look at violations themselves now.  We'll need to think more carefully about what we're measuring, since most restaurants have many inspections with possibly multiple violations per inspection, or long stretches of inspections with no violations.

There are many ways to deal with this normalization issue, but we'll go with a fairly straightforward one: dividing the number of violations by the length of time (in years) the restaurant has been open.  As a proxy for the length, we'll look at the difference between the oldest and newest inspection date, treating anything less than 30 days as 30 days (to account for those that were only inspected once, we'll assume everything was open for at least a month).

Since there are so many restaurants, we'll group them by cuisine and do a weighted average by computing 

    (total violations for a cuisine) / (total restaurant-years for that cuisine)

Return a list of 75 tuples of the form

    (cuisine name, reports per restaurant-year)
    
for cuisines with at least 100 violations total, ordered by increasing number of reports per restaurant-year
    
**Note:** This isn't the only way to normalize things.  How would other ways affect the computation?  If you similarly wanted to compute an average score by cuisine, how might you go about doing that?
    
**Checkpoint:**
- Total entries from valid cuisines: 522,410

In [ ]:
%%sql 
DROP TABLE IF EXISTS ages;
CREATE TABLE ages AS
SELECT CODEDESC, SUM(age) AS age FROM
(SELECT CODEDESC, (CASE WHEN  (JULIANDAY(MAX(inspdate))-JULIANDAY(MIN(inspdate)))< 30 THEN 30
             ELSE  (JULIANDAY(MAX(inspdate))-JULIANDAY(MIN(inspdate)))
        END)AS age FROM temp_webextract 
INNER JOIN cuisine
ON temp_webextract.CUISINECODE = cuisine.CUISINECODE
GROUP BY CODEDESC, camis)
GROUP BY CODEDESC;



In [ ]:
%%sql
 
ORDER BY  (num_viol/(age*360)) 

In [ ]:
Result = _

In [ ]:
score_by_cuisine = [(y, z) for y,z in Result]

In [ ]:
len(score_by_cuisine)

In [ ]:
# score_by_cuisine = [("French", 20.3550686378036)] * 75

grader.score('sql__score_by_cuisine', score_by_cuisine)

## Question 5: Specific violations by cuisine

Which cuisines tend to have a disproportionate number of what which violations? Answering this question isn't easy because you have to think carefully about normalizations.

1. More popular cuisine categories will tend to have more violations just because they represent more restaurants.
2. Similarly, some violations are more common.  For example, knowing that "Equipment not easily movable or sealed to floor" is a common violation for Chinese restaurants is not particularly helpful when it is a common violation for all restaurants.

The right quantity is to look at is the conditional probability of a specific type of violation given a specific cuisine type and divide it by the unconditional probability of the violation for the entire population. Taking this ratio gives the right answer.  Return the 20 highest ratios of the form:

    ((cuisine, violation), ratio, count)

**Hint:**
1. You might want to check out this [Stack Overflow post](http://stackoverflow.com/questions/972877/calculate-frequency-using-sql).
2. The definition of a violation changes with time.  For example, 10A can mean two different things "Toilet facility not maintained ..." or "Vermin or other live animal present ..." when things were prior to 2003. To deal with this, you should limit your analysis to violation codes with end date after Jan 1, 2014. (This end date refers to the validity time ranges in `Violation.txt`).
3. The ratios don't mean much when the number of violations of a given type and for a specific category are not large (why not?).  Be sure to filter these out.  We chose 100 as our cutoff.

**Checkpoint:**
- Top 20 ratios mean: 2.360652529900757

In [ ]:
%%sql
DROP TABLE IF EXISTS violation_endate;

CREATE TABLE violation_endate AS 
SELECT A.CUISINECODE AS CUISINECODE, A.VIOLCODE AS VIOLCODE ,B.ENDDATE AS ENDDATE,
A.COUNT0 AS COUNT0 FROM
(SELECT CUISINECODE, VIOLCODE, COUNT(*) AS COUNT0 From temp_webextract
Group By CUISINECODE, VIOLCODE
) AS A
INNER JOIN (SELECT VIOLATIONCODE,ENDDATE FROM violations) AS B
 ON A.VIOLCODE = B.VIOLATIONCODE
WHERE ENDDATE>'2014-01-1 00:00:00'


In [ ]:
%%sql
SELECT * FROM violation_endate

In [ ]:
%%sql
DROP TABLE IF EXISTS cond_prob;

CREATE TABLE cond_prob AS 
SELECT  A.CUISINECODE AS CUISINECODE, A.VIOLCODE AS VIOLCODE , A.COUNT0 * 1.0 / B.COUNT2 As Freq,
A.COUNT0 AS COUNT0 From    
(Select CUISINECODE, VIOLCODE,COUNT0 
From   violation_endate 
Group By CUISINECODE, VIOLCODE
) As A
Inner Join (Select CUISINECODE, COUNT(*) As COUNT2
            From   temp_webextract 
            Group By CUISINECODE
            ) As B
            On A.CUISINECODE = B.CUISINECODE

In [ ]:
%%sql 
SELECT * FROM cond_prob
LIMIT 5

In [ ]:
%%sql
DROP TABLE IF EXISTS new_webextract;

CREATE TABLE new_webextract AS
SELECT A.CUISINECODE AS CUISINECODE, A.VIOLCODE AS VIOLCODE ,B.ENDDATE AS ENDDATE FROM
( SELECT CUISINECODE, VIOLCODE FROM temp_webextract) AS A
INNER JOIN (SELECT VIOLATIONCODE,ENDDATE FROM violations) AS B
ON A.VIOLCODE = B.VIOLATIONCODE
WHERE ENDDATE>'2014-01-1';

In [ ]:
%%sql 
SELECT * FROM new_webextract
LIMIT 5

In [ ]:
%%sql
DROP TABLE IF EXISTS vio_table;
CREATE TABLE vio_table AS
Select A.VIOLCODE AS VIOLCODE,A.COUNT3 AS COUNT3, B.vio_count As TOTAL From    
(Select VIOLCODE,COUNT(*) AS COUNT3
From   new_webextract 
Group By VIOLCODE
) As A
Inner Join (
Select Count(*) As vio_count
From   new_webextract 
) As B

In [ ]:
%%sql
DROP TABLE IF EXISTS prob_vio;
CREATE TABLE prob_vio AS
SELECT VIOLCODE,COUNT3*1.0/TOTAL AS Prob_vio FROM vio_table GROUP BY VIOLCODE

In [ ]:
%%sql

DROP TABLE IF EXISTS final_table;

CREATE TABLE final_table AS
Select A.CUISINECODE AS CUISINECODE, B.VIOLCODE AS VIOLCODE, A.Freq/B.Prob_vio As RATIO, A.COUNT0 AS COUNT0 From
(Select CUISINECODE,VIOLCODE,Freq,COUNT0 
From cond_prob 
WHERE COUNT0>100) As A
Inner Join (Select VIOLCODE, Prob_vio
From   prob_vio 
) As B
On A.VIOLCODE = B.VIOLCODE

In [ ]:
%%sql
SELECT * FROM final_table


In [ ]:
%%sql
DROP TABLE IF EXISTS final;
CREATE TABLE final AS
SELECT * FROM final_table
JOIN cuisine
ON final_table.CUISINECODE = cuisine.CUISINECODE


In [ ]:
%%sql
SELECT * FROM final

In [ ]:
%%sql 
DROP TABLE IF EXISTS viol;
CREATE TABLE viol AS
SELECT * FROM violations
WHERE enddate > '2014-01-01'


In [ ]:
%%sql
SELECT * FROM viol

In [ ]:
%%sql
DROP TABLE IF EXISTS new_final;
CREATE TABLE new_final AS
SELECT CODEDESC, violationdesc, RATIO, COUNT0  FROM final
LEFT OUTER JOIN viol
ON final.violcode = viol.violationcode


In [ ]:
%%sql 
SELECT * FROM new_final

In [ ]:
%%sql
SELECT * FROM new_final
ORDER BY RATIO DESC
LIMIT 20

In [ ]:
RESULT = _

In [ ]:
violation_by_cuisine = [((x,y), w,z) for x,y,w,z in RESULT]

In [ ]:
violation_by_cuisine

In [ ]:
# violation_by_cuisine = [
#     (("Café/Coffee/Tea",
#       "Toilet facility not maintained and provided with toilet paper; "
#       "waste receptacle and self-closing door."),
#      1.87684775827172, 315)] * 20

grader.score('sql__violation_by_cuisine', violation_by_cuisine)

*Copyright &copy; 2020 Pragmatic Institute. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*